In [20]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import os

In [21]:
file = open("/research/video_metaphor/LLaVA/results/llava13-b/vmcd_ispy/24f_4parts/5_ep/answers_t_0_2_test.jsonl", "r")
questions = [json.loads(q) for q in file]
print(len(questions))

750


In [22]:
questions[0]

{'question_id': 0,
 'prompt': 'What metaphor can best describe the video?',
 'video_id': 'jmNrvlQ73pU',
 'true_caption': 'the burger is as comforting as relationship',
 'text': 'the toys are as cute as the girl.',
 'answer_id': 'WpKhrFYFcFVNBv3gLmomGE',
 'model_id': 'llava1.5-13b-24f-4part_checkpoint-375',
 'metadata': {}}

In [23]:
questions[0]
# true_captions = [ q["true_caption"].split(":")[1].strip() for q in questions]
true_captions = [ q["true_caption"] for q in questions]
pred_captions = [ q["text"].split("\n")[0] for q in questions]
video_ids = [ q["video_id"] for q in questions]

In [24]:
video_ids[2]

'jmNrvlQ73pU'

In [25]:
predicted_df = pd.DataFrame({"video_id": video_ids, "true_caption":true_captions, "pred_captions": pred_captions})
print(len(predicted_df))
predicted_df.head()

750


,video_id,true_caption,pred_captions
0,jmNrvlQ73pU,the burger is as comforting as relationship,the toys are as cute as the girl.
1,jmNrvlQ73pU,a burger is as much a source of satisfaction a...,the toy is as cute as the girl.
2,jmNrvlQ73pU,the burger is as a bond as kids relationship,the chocolate is as sweet as the relationship ...
3,D93S3UVU8pk,the game is as fun as the cartoon,the drink is as cool as a polar bear.
4,D93S3UVU8pk,the game is as interesting as a cartoon.,the game is as exciting as a superhero.


In [26]:
# cap_list = {}
reference_df = predicted_df.groupby('video_id')['true_caption'].apply(list).reset_index(name="reference_captions")
reference_df



,video_id,reference_captions
0,0-U4Yr9UNBo,[the enormous phone is as good as the minor ph...
1,0B79RkBxSgU,[the actor is not as refreshing as a soft drin...
2,0XSNrKgA94g,"[chocolate is as light as a man who can fly., ..."
3,0cP-qr1JUBI,[choosing real estate agent is as important as...
4,10okyjMjlH8,[the game is as flexible as a world with creat...
...,...,...
245,z1zeaOK2atg,"[soap is as beautiful as diversity., soap is a..."
246,z62vD5BDDUE,[a mutual fund is as visionary as teaching a k...
247,zKQbsrRutvE,"[the burger is as original as irish people, th..."
248,zSM6sAtyvyg,"[sippy cups are as cool as husbands., sippy cu..."


In [27]:
merged_df = pd.merge(predicted_df, reference_df, on=['video_id'])
print(len(merged_df))
merged_df.head()

750


,video_id,true_caption,pred_captions,reference_captions
0,jmNrvlQ73pU,the burger is as comforting as relationship,the toys are as cute as the girl.,"[the burger is as comforting as relationship, ..."
1,jmNrvlQ73pU,a burger is as much a source of satisfaction a...,the toy is as cute as the girl.,"[the burger is as comforting as relationship, ..."
2,jmNrvlQ73pU,the burger is as a bond as kids relationship,the chocolate is as sweet as the relationship ...,"[the burger is as comforting as relationship, ..."
3,D93S3UVU8pk,the game is as fun as the cartoon,the drink is as cool as a polar bear.,"[the game is as fun as the cartoon, the game i..."
4,D93S3UVU8pk,the game is as interesting as a cartoon.,the game is as exciting as a superhero.,"[the game is as fun as the cartoon, the game i..."


In [28]:
reference_captions = merged_df["reference_captions"].tolist()
reference_captions[0]

['the burger is as comforting as relationship',
 'a burger is as much a source of satisfaction as a relationship.',
 'the burger is as a bond as kids relationship']

In [29]:
# from nltk.translate.bleu_score import corpus_bleu
# # references = [[cap.split()] for cap in true_captions]
# references = [[cap.split() for cap in references] for references in reference_captions]
# candidates = [cap.split() for cap in pred_captions]
# score = corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25))
# print(score)
# score = corpus_bleu(references, candidates, weights=(1, 0,0,0))
# print(score)

In [30]:
# # from rouge_score import rouge_scorer

# # scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# # scores = scorer.score('The quick brown fox jumps over the lazy dog',
# #                       'The quick brown dog jumps on the log.')

# import evaluate
# rouge = evaluate.load('rouge')

# true_captions_r = [[c] for c in true_captions]
# results = rouge.compute(predictions=pred_captions, references=true_captions_r)
# print(results)


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', device="cuda")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device="cuda:4")

In [31]:
import re
def compute_similarity_between_parts(sentence, f1_score):
    pattern = r'as \w+ as'
    parts = re.split(pattern, sentence)

    # Remove any leading or trailing whitespace from the parts
    parts = [part.strip() for part in parts if part.strip()]
    
    
    if len(parts) < 2:
        cosine_score = float(1.0)

    else:

    #Compute embedding for both primary and secondary concept
        # Removing last word from primary concept as it could be is or are
        parts[0] = " ".join(parts[0].split()[:-1])
        embeddings1 = model.encode(parts[0], convert_to_tensor=True)
        embeddings2 = model.encode(parts[1], convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_score = util.cos_sim(embeddings1, embeddings2)[0][0]

    cosine_distance = 1 - cosine_score
    return (cosine_distance * f1_score)

def compute_similarity_between_parts_without_f1(sentence):
    pattern = r'as \w+ as'
    parts = re.split(pattern, sentence)

    # Remove any leading or trailing whitespace from the parts
    parts = [part.strip() for part in parts if part.strip()]
    
    if len(parts) < 2:
        # cosine_score = float(1.0)
        return float(1.0)

    else:

    #Compute embedding for both primary and secondary concept
        # Removing last word from primary concept as it could be is or are
        parts[0] = " ".join(parts[0].split()[:-1])

        embeddings1 = model.encode(parts[0], convert_to_tensor=True)
        embeddings2 = model.encode(parts[1], convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_score = util.cos_sim(embeddings1, embeddings2)[0][0]

        return cosine_score

    

def split_to_parts(sentence):
    pattern = r'as \w+ as'
    parts = re.split(pattern, sentence)

    # Remove any leading or trailing whitespace from the parts
    parts = [part.strip() for part in parts if part.strip()]
    # Removing last word from primary concept as it could be is or are
    parts[0] = " ".join(parts[0].split()[:-1])
    if len(parts) == 1:
        parts.append("")
    return parts


def compute_similarity(true_caption, predicted_caption):
    
    #Compute embedding for both primary and secondary concept
    embeddings1 = model.encode(true_caption, convert_to_tensor=True)
    embeddings2 = model.encode(predicted_caption, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_score = util.cos_sim(embeddings1, embeddings2)

    return float(cosine_score[0][0])

In [ ]:
!pip install pycocoevalcap

In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master

In [32]:
# from nlgeval.pycocoevalcap.bleu.bleu import Bleu
# from nlgeval.pycocoevalcap.rouge.rouge import Rouge
# from nlgeval.pycocoevalcap.cider.cider import Cider
# from nlgeval.pycocoevalcap.meteor.meteor import Meteor

from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor

In [33]:
def score(ref, hypo):
    scorers = [
        (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
        (Rouge(), "ROUGE_L"),
        (Cider(), "CIDEr")
    ]
    final_scores = {}
    all_scores = {}
    for scorer, method in scorers:
        score, scores = scorer.compute_score(ref, hypo)

        if type(score) == list:
            for m, s in zip(method, score):
                final_scores[m] = s
            for m, s in zip(method, scores):
                all_scores[m] = s
        else:
            final_scores[method] = score
            all_scores[method] = scores

    return final_scores, all_scores


def compute(ref, cand, get_scores=True):
    # make dictionary
    hypo = {}
    for i, caption in enumerate(cand):
        hypo[i] = [caption]
    truth = {}
    # for i, caption in enumerate(ref):
    #     truth[i] = [caption]
    for i, caption in enumerate(ref):
        truth[i] = caption

    # compute bleu score
    final_scores = score(truth, hypo)

    #     print out scores
    print('Bleu_1:\t ;', final_scores[0]['Bleu_1']*100)
    print('Bleu_2:\t ;', final_scores[0]['Bleu_2']*100)
    print('Bleu_3:\t ;', final_scores[0]['Bleu_3']*100)
    print('Bleu_4:\t ;', final_scores[0]['Bleu_4']*100)
    print('ROUGE_L: ;', final_scores[0]['ROUGE_L']*100)
    print('CIDEr:\t ;', final_scores[0]['CIDEr']*100)

    if get_scores:
        return final_scores

In [34]:

# res = compute(true_captions, pred_captions, get_scores=True)
res = compute(reference_captions, pred_captions, get_scores=True)

# output = score(true_captions, pred_captions)

{'testlen': 6223, 'reflen': 6140, 'guess': [6223, 5473, 4723, 3973], 'correct': [3404, 1386, 364, 165]}
ratio: 1.013517915309281
Bleu_1:	 ; 54.700305318969185
Bleu_2:	 ; 37.21891995226695
Bleu_3:	 ; 22.019307522630893
Bleu_4:	 ; 14.510891253059183
ROUGE_L: ; 50.59025726463086
CIDEr:	 ; 22.66865495813586


In [ ]:
tc = ["I met batman", "I am superman"]
pc = ["I am batman", "I met superman"]
res = compute(tc, pc, get_scores=True)

In [ ]:
!pip install bert_score

In [35]:
from evaluate import load
bertscore = load("bertscore")
# predictions = ["hello world", "general kenobi"]
# references = ["hello world", "general kenobi"]
results = bertscore.compute(predictions=pred_captions, references=reference_captions, model_type="microsoft/deberta-xlarge-mnli", device="cuda:4")
# results = bertscore.compute(predictions=pred_captions, references=true_captions, lang="en")
# print(results)
precision = sum(results['precision'])/len(results['precision'])
recall = sum(results['recall'])/len(results['recall'])
f1 = sum(results['f1'])/len(results['f1'])

print(precision)
print(recall)
print(f1)                           

0.7535977378686269
0.7252129632631937
0.7356398804187775


In [36]:
# primary_secondary_concepts = [split_to_parts(sent) for sent in tqdm(pred_captions)]
# # print(primary_secondary_concepts[0:5])

# primary_concepts = [item[0] for item in primary_secondary_concepts]
# secondary_concepts = [item[1] for item in primary_secondary_concepts]

# # print(primary_concepts[0:5])
# # print(secondary_concepts[0:5])

# results = bertscore.compute(predictions=secondary_concepts, references=primary_concepts, model_type="microsoft/deberta-xlarge-mnli")
# # print(results)
# precision = sum(results['precision'])/len(results['precision'])
# recall = sum(results['recall'])/len(results['recall'])
# f1 = sum(results['f1'])/len(results['f1'])

# print(precision)
# print(recall)
# print(f1)


In [37]:
# ACS
cosine_sim_scores = [compute_similarity_between_parts_without_f1(sent) for sent in tqdm(pred_captions)]
avg_sim_score = sum(cosine_sim_scores)/len(cosine_sim_scores)
print(avg_sim_score)


# ACD

cosine_scores = [compute_similarity_between_parts(sent, f1_score) for sent,f1_score in tqdm(zip(pred_captions, results['f1']) )]
# print(cosine_scores[0:5])
avg_score = sum(cosine_scores)/len(cosine_scores)
different_statements = [score for score in cosine_scores if score < 0.5]
print(avg_score, len(different_statements)/len(pred_captions))



100%|██████████| 750/750 [00:14<00:00, 51.35it/s]


tensor(0.2856, device='cuda:4')


750it [00:14, 51.30it/s]


tensor(0.5258, device='cuda:4') 0.37733333333333335


In [ ]:
pred_primary_secondary_concepts = [split_to_parts(sent) for sent in tqdm(pred_captions)]
# print(primary_secondary_concepts[0:5])
pred_primary_concepts = [item[0] for item in pred_primary_secondary_concepts]




true_primary_secondary_concepts = [split_to_parts(sent) for sent in tqdm(true_captions)]
true_primary_concepts = [item[0] for item in true_primary_secondary_concepts]



In [ ]:
import nltk

def unique_words(captions):
    tokens = []

    for caption in captions:
        words = nltk.word_tokenize(caption)
        tokens += words

    print(len(tokens))
    print(len(list(set(tokens))) / len(pred_captions))

unique_words(pred_primary_concepts)
unique_words(true_primary_concepts)

In [ ]:
pred_primary_concepts[0:15]

In [ ]:
true_primary_concepts[0:15]